In [106]:
#Importing necessary packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import lightgbm as lgb
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from matplotlib import style
from datetime import datetime, timedelta
import itertools

style.use('ggplot')

In [107]:
#Importing data and setting saving directory

path = "/home/marius/Documents/MT Data/Extensive_incident_list_full.csv"
save_path = "/home/marius/Documents/MTD_Plots/Full data plots"

data = pd.read_csv(path, encoding = "ISO-8859-1")

/home/marius/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3044: DtypeWarning: Columns (5,8,9,13,16,17,19,23,24,25,27,28,29,30,31,32,33,34,38,39,41,42,45,49,50,51,52,53,54,57,58,59,60,61,63,65,66,68,69,70,71,74,77,79,80,86,88,90,91,92,94,96,97,105,126,127,128,129,130,131,132,138,140) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [108]:
len(list(data))

141

## Cleaning data

In [109]:
#Removing columns unneccesarry for regression task
unneccessary_1 = list(data)[:10]
unneccessary_2 = list(data)[11:13]
unneccessary_3 = list(data)[14:16]
unneccessary_4 = [list(data)[17]]
unneccessary_5 = list(data)[22:32]
unneccessary_6 = list(data)[-30:-8]

unneccessary = list(itertools.chain(unneccessary_1, unneccessary_2, unneccessary_3, unneccessary_4, unneccessary_5, unneccessary_6))
data = data.drop(unneccessary, axis=1)

In [110]:
#

In [111]:
#Unneccessary until I can speak to Kaipeng about the columns
remove_list = ['sys_mod_count',
 'u_unit',
 'sys_class_name',
 'sys_tags',
 'u_supplier',
 'subcategory',
 'correlation_id',
 'u_sap_data',
 'u_sap_component',
 'u_resolved_by',
 'u_resolved_at',
 'calendar_stc',
 'u_reopened_counter',
 'reassignment_count',
 'u_reason_for_waiting',
 'u_qualified_by_it',
 'u_project_related',
 'problem_id',
 'parent',
 'order',
 'opened_by',
 'opened_at',
 'notify',
'made_sla',
 'location',
 'reopened_by',
 'reopened_time',
 'knowledge',
 'u_incident_id',
'u_watch_list_it',
 'u_hypercare_related',
 'u_group_visibility',
 'group_list',
 'follow_up',
 'u_external_reference',
 'expected_start',
'calendar_duration',
 'due_date',
 'sys_domain',
 'u_description_html',
 'description',
 'delivery_task',
 'delivery_plan',
 'sys_created_by',
 'correlation_display',
 'contact_type',
 'u_confirm_priority_1',
 'cmdb_ci',
'u_comment_added_it',
 'u_comment_added',
 'closed_by',
'close_notes',
 'close_code',
 'rfc',
 'caused_by',
 'business_service',
 'business_stc',
 'business_duration',
 'approval_set',
 'approval_history',
 'approval',
 'additional_assignee_list',
 'comments',
 'work_start',
 'work_end',
 'activity_due',
 'caller_id',
 'assignment_group',
 'assigned_to',
 'assigned_to.email',
 'active',
'caller_id.company',
'caller_id.country',
 'caller_id.department',
 'caller_id.sys_domain',
 'caller_id.location',
 'company.country',
 'company.u_location.full_name',
 'company.u_location.time_zone',
 'company.rank_tier',
 'company.primary',
 'Unnamed: 140']

In [112]:
data = data.drop(remove_list, axis=1)

In [113]:
data.head()

urgency u_major_incident   impact escalation       sys_created_on company  \
0  3 - Low            FALSE  3 - Low     Normal  06-06-2011 23:07:16     201   
1  3 - Low            FALSE  3 - Low     Normal  07-06-2011 17:07:28     201   
2  3 - Low            FALSE  3 - Low     Normal  13-06-2011 16:59:18     215   
3  3 - Low            FALSE  3 - Low     Normal  14-06-2011 14:36:03     215   
4  3 - Low            FALSE  3 - Low     Normal  14-06-2011 15:35:51     201   

             closed_at category      priority  \
0  07-07-2011 06:00:05  Failure  5 - Planning   
1  09-08-2011 06:00:10  Service  5 - Planning   
2  20-06-2011 06:00:00  Failure  5 - Planning   
3  15-08-2011 06:00:09  Failure  5 - Planning   
4  13-07-2011 06:00:06  Failure  5 - Planning   

  caller_id.u_communication_language company.u_location.country  \
0                                 DE                Switzerland   
1                                 DE                Switzerland   
2                                 EN                Switzerland   
3                                 EN                Switzerland   
4                                 DE                Switzerland   

  company.u_location.zip  
0                   8606  
1                   8606  
2                   8606  
3                   8606  
4                   8606

In [114]:
data = data.dropna()
data = data.reset_index(drop=True)

In [115]:
data.isnull().sum()

urgency                               0
u_major_incident                      0
impact                                0
escalation                            0
sys_created_on                        0
company                               0
closed_at                             0
category                              0
priority                              0
caller_id.u_communication_language    0
company.u_location.country            0
company.u_location.zip                0
dtype: int64

In [116]:
data.head()

urgency u_major_incident   impact escalation       sys_created_on company  \
0  3 - Low            FALSE  3 - Low     Normal  06-06-2011 23:07:16     201   
1  3 - Low            FALSE  3 - Low     Normal  07-06-2011 17:07:28     201   
2  3 - Low            FALSE  3 - Low     Normal  13-06-2011 16:59:18     215   
3  3 - Low            FALSE  3 - Low     Normal  14-06-2011 14:36:03     215   
4  3 - Low            FALSE  3 - Low     Normal  14-06-2011 15:35:51     201   

             closed_at category      priority  \
0  07-07-2011 06:00:05  Failure  5 - Planning   
1  09-08-2011 06:00:10  Service  5 - Planning   
2  20-06-2011 06:00:00  Failure  5 - Planning   
3  15-08-2011 06:00:09  Failure  5 - Planning   
4  13-07-2011 06:00:06  Failure  5 - Planning   

  caller_id.u_communication_language company.u_location.country  \
0                                 DE                Switzerland   
1                                 DE                Switzerland   
2                                 EN                Switzerland   
3                                 EN                Switzerland   
4                                 DE                Switzerland   

  company.u_location.zip  
0                   8606  
1                   8606  
2                   8606  
3                   8606  
4                   8606

In [117]:
#Removing lettering from what is supposed tobe a numerical value
ind_val=[]
num_col = ["urgency", "impact", "priority"]
for col in (num_col):
    print(col)
    for i in range(len(data)):
        val = data.iloc[i][col]
        if isinstance(val, int):
            data.set_value([i], col, val)
            continue
        try: 
            val = int(val.split()[0])
            data.set_value([i], col, val)
            continue
        except:
            #print("String where there shouldn be at index", i)
            ind_val += [i]
            continue
        
    print("Finito with " + col)
print("Finito total")

urgency


/home/marius/.local/lib/python3.6/site-packages/ipykernel_launcher.py:13: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  del sys.path[0]


Finito with urgency
impact
Finito with impact
priority
Finito with priority
Finito total


In [118]:
data

urgency u_major_incident impact escalation       sys_created_on company  \
0           3            FALSE      3     Normal  06-06-2011 23:07:16     201   
1           3            FALSE      3     Normal  07-06-2011 17:07:28     201   
2           3            FALSE      3     Normal  13-06-2011 16:59:18     215   
3           3            FALSE      3     Normal  14-06-2011 14:36:03     215   
4           3            FALSE      3     Normal  14-06-2011 15:35:51     201   
5           3            FALSE      2     Normal  15-06-2011 19:55:02     201   
6           3            FALSE      3     Normal  16-06-2011 17:26:47     201   
7           3            FALSE      3     Normal  17-06-2011 20:13:16     215   
8           3            FALSE      2     Normal  17-06-2011 22:42:40     201   
9           3            FALSE      3     Normal  21-06-2011 15:56:38     201   
10          3            FALSE      3     Normal  21-06-2011 18:19:35     215   
11          3            FALSE      3     Normal  21-06-2011 22:26:38     201   
12          3            FALSE      2     Normal  22-06-2011 17:14:09     201   
13          3            FALSE      2     Normal  24-06-2011 14:01:11     201   
14          3            FALSE      2     Normal  27-06-2011 15:34:11     201   
15          3            FALSE      1     Normal  27-06-2011 15:38:12     201   
16          3            FALSE      2     Normal  27-06-2011 15:46:02     201   
17          3            FALSE      3     Normal  27-06-2011 18:18:37     215   
18          1            FALSE      2     Normal  27-06-2011 21:07:42     201   
19          3            FALSE      2     Normal  28-06-2011 14:23:30     201   
20          3            FALSE      3     Normal  28-06-2011 21:00:41     201   
21          2            FALSE      1     Normal  28-06-2011 22:51:58     201   
22          3            FALSE      3     Normal  29-06-2011 12:32:38     214   
23          2            FALSE      2     Normal  30-06-2011 00:11:05    1007   
24          3            FALSE      3     Normal  30-06-2011 13:28:04     215   
25          3            FALSE      3     Normal  30-06-2011 19:46:26     215   
26          3            FALSE      3     Normal  30-06-2011 20:21:06    2801   
27          3            FALSE      3     Normal  30-06-2011 20:35:18    2805   
28          3            FALSE      3     Normal  30-06-2011 20:41:22    2805   
29          3            FALSE      3     Normal  30-06-2011 20:54:10     214   
...       ...              ...    ...        ...                  ...     ...   
31531       2            False      2     Normal  08-07-2019 09:32:45    2805   
31532       3            False      3     Normal  08-07-2019 11:17:47    1401   
31533       2            False      3     Normal  08-07-2019 13:59:46     101   
31534       2            False      3     Normal  08-07-2019 16:05:08     101   
31535       3            False      3     Normal  08-07-2019 17:38:09     201   
31536       3            False      3     Normal  08-07-2019 18:22:38    4602   
31537       2            False      2     Normal  08-07-2019 19:24:45     201   
31538       3            False      3     Normal  08-07-2019 19:59:21     501   
31539       3            False      3     Normal  08-07-2019 22:27:59    4602   
31540       3            False      3     Normal  08-07-2019 22:56:11    1005   
31541       3            False      3     Normal  09-07-2019 11:29:03    1403   
31542       3            False      3     Normal  09-07-2019 13:19:40    4005   
31543       3            False      1     Normal  09-07-2019 14:11:55    4602   
31544       3            False      3     Normal  09-07-2019 14:59:07     201   
31545       3            False      3     Normal  09-07-2019 15:04:37    4602   
31546       3            False      3     Normal  09-07-2019 15:15:40    4602   
31547       3            False      3     Normal  09-07-2019 17:56:53     701   
31548       3            False      3

In [119]:
### Dropping rows with faulty urgency, impact, and priority data
ind_val = list(set(ind_val))
data = data.drop(ind_val, axis=1)

In [120]:
#Turning string type booleans into bool type booleans
ind_val=[]
for i in range(len(data)):
    val = data.iloc[i]["u_major_incident"]
    if val == "FALSE":
        data.set_value(i, "u_major_incident", False)
    elif val == "TRUE":
        data.set_value(i, "u_major_incident", True)
    elif isinstance(val, bool):
        continue
    else:
        print(val)
        ind_val += [i]
list(set(ind_val))
data = data.drop(ind_val)
data = data.reset_index(drop=True)

/home/marius/.local/lib/python3.6/site-packages/ipykernel_launcher.py:6: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  
/home/marius/.local/lib/python3.6/site-packages/ipykernel_launcher.py:8: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  


In [121]:
data.head()

urgency u_major_incident impact escalation       sys_created_on company  \
0       3            False      3     Normal  06-06-2011 23:07:16     201   
1       3            False      3     Normal  07-06-2011 17:07:28     201   
2       3            False      3     Normal  13-06-2011 16:59:18     215   
3       3            False      3     Normal  14-06-2011 14:36:03     215   
4       3            False      3     Normal  14-06-2011 15:35:51     201   

             closed_at category priority caller_id.u_communication_language  \
0  07-07-2011 06:00:05  Failure        5                                 DE   
1  09-08-2011 06:00:10  Service        5                                 DE   
2  20-06-2011 06:00:00  Failure        5                                 EN   
3  15-08-2011 06:00:09  Failure        5                                 EN   
4  13-07-2011 06:00:06  Failure        5                                 DE   

  company.u_location.country company.u_location.zip  
0                Switzerland                   8606  
1                Switzerland                   8606  
2                Switzerland                   8606  
3                Switzerland                   8606  
4                Switzerland                   8606

In [122]:
#Removing wrong entries in escalation
ok_entries = ["Normal", "High", "Moderate", "Overdue"]
data = data[data.escalation.isin(ok_entries)]
data = data.reset_index(drop=True)

In [123]:
data.head()

urgency u_major_incident impact escalation       sys_created_on company  \
0       3            False      3     Normal  06-06-2011 23:07:16     201   
1       3            False      3     Normal  07-06-2011 17:07:28     201   
2       3            False      3     Normal  13-06-2011 16:59:18     215   
3       3            False      3     Normal  14-06-2011 14:36:03     215   
4       3            False      3     Normal  14-06-2011 15:35:51     201   

             closed_at category priority caller_id.u_communication_language  \
0  07-07-2011 06:00:05  Failure        5                                 DE   
1  09-08-2011 06:00:10  Service        5                                 DE   
2  20-06-2011 06:00:00  Failure        5                                 EN   
3  15-08-2011 06:00:09  Failure        5                                 EN   
4  13-07-2011 06:00:06  Failure        5                                 DE   

  company.u_location.country company.u_location.zip  
0                Switzerland                   8606  
1                Switzerland                   8606  
2                Switzerland                   8606  
3                Switzerland                   8606  
4                Switzerland                   8606

In [124]:
#Calculating time spent on ticket, adding new column, and dropping old ones

def date_func(datestring):
    FMT = '%d-%m-%Y %H:%M:%S'
    t1 = datetime.strptime(datestring, FMT)
    return t1

#Creating new column with time spent to complete ticket
lis = ['sys_created_on', 'closed_at']
for col in lis:
    data[col] = data[col].map(lambda col: date_func(col))

In [125]:
data.head()

urgency u_major_incident impact escalation      sys_created_on company  \
0       3            False      3     Normal 2011-06-06 23:07:16     201   
1       3            False      3     Normal 2011-06-07 17:07:28     201   
2       3            False      3     Normal 2011-06-13 16:59:18     215   
3       3            False      3     Normal 2011-06-14 14:36:03     215   
4       3            False      3     Normal 2011-06-14 15:35:51     201   

            closed_at category priority caller_id.u_communication_language  \
0 2011-07-07 06:00:05  Failure        5                                 DE   
1 2011-08-09 06:00:10  Service        5                                 DE   
2 2011-06-20 06:00:00  Failure        5                                 EN   
3 2011-08-15 06:00:09  Failure        5                                 EN   
4 2011-07-13 06:00:06  Failure        5                                 DE   

  company.u_location.country company.u_location.zip  
0                Switzerland                   8606  
1                Switzerland                   8606  
2                Switzerland                   8606  
3                Switzerland                   8606  
4                Switzerland                   8606

In [126]:
data["Completion Time"] = data['closed_at'] - data['sys_created_on'] 
data.head()

urgency u_major_incident impact escalation      sys_created_on company  \
0       3            False      3     Normal 2011-06-06 23:07:16     201   
1       3            False      3     Normal 2011-06-07 17:07:28     201   
2       3            False      3     Normal 2011-06-13 16:59:18     215   
3       3            False      3     Normal 2011-06-14 14:36:03     215   
4       3            False      3     Normal 2011-06-14 15:35:51     201   

            closed_at category priority caller_id.u_communication_language  \
0 2011-07-07 06:00:05  Failure        5                                 DE   
1 2011-08-09 06:00:10  Service        5                                 DE   
2 2011-06-20 06:00:00  Failure        5                                 EN   
3 2011-08-15 06:00:09  Failure        5                                 EN   
4 2011-07-13 06:00:06  Failure        5                                 DE   

  company.u_location.country company.u_location.zip  Completion Time  
0                Switzerland                   8606 30 days 06:52:49  
1                Switzerland                   8606 62 days 12:52:42  
2                Switzerland                   8606  6 days 13:00:42  
3                Switzerland                   8606 61 days 15:24:06  
4                Switzerland                   8606 28 days 14:24:15

In [127]:
data["Completion Time"] = data["Completion Time"].map(lambda col: float(col.total_seconds()/3600))
data.head()

urgency u_major_incident impact escalation      sys_created_on company  \
0       3            False      3     Normal 2011-06-06 23:07:16     201   
1       3            False      3     Normal 2011-06-07 17:07:28     201   
2       3            False      3     Normal 2011-06-13 16:59:18     215   
3       3            False      3     Normal 2011-06-14 14:36:03     215   
4       3            False      3     Normal 2011-06-14 15:35:51     201   

            closed_at category priority caller_id.u_communication_language  \
0 2011-07-07 06:00:05  Failure        5                                 DE   
1 2011-08-09 06:00:10  Service        5                                 DE   
2 2011-06-20 06:00:00  Failure        5                                 EN   
3 2011-08-15 06:00:09  Failure        5                                 EN   
4 2011-07-13 06:00:06  Failure        5                                 DE   

  company.u_location.country company.u_location.zip  Completion Time  
0                Switzerland                   8606       726.880278  
1                Switzerland                   8606      1500.878333  
2                Switzerland                   8606       157.011667  
3                Switzerland                   8606      1479.401667  
4                Switzerland                   8606       686.404167

In [128]:
data = data.drop(lis, axis=1)
data = data.reset_index(drop=True)

In [129]:
data.head()

urgency u_major_incident impact escalation company category priority  \
0       3            False      3     Normal     201  Failure        5   
1       3            False      3     Normal     201  Service        5   
2       3            False      3     Normal     215  Failure        5   
3       3            False      3     Normal     215  Failure        5   
4       3            False      3     Normal     201  Failure        5   

  caller_id.u_communication_language company.u_location.country  \
0                                 DE                Switzerland   
1                                 DE                Switzerland   
2                                 EN                Switzerland   
3                                 EN                Switzerland   
4                                 DE                Switzerland   

  company.u_location.zip  Completion Time  
0                   8606       726.880278  
1                   8606      1500.878333  
2                   8606       157.011667  
3                   8606      1479.401667  
4                   8606       686.404167

In [130]:
data = data.drop("company.u_location.country", axis=1)

In [131]:
data = data.drop("u_major_incident", axis=1)

In [132]:
#Changing data types to ints for numerical values

lis = ['urgency', 'impact', "priority"]
for col in lis:
    data[col] = data[col].map(lambda col: int(col))

In [133]:
#1% of the len(data) is 315.61, so any categorical data less than this is considered an outlier

In [134]:
data['caller_id.u_communication_language'].value_counts()

EN    21556
DE     5634
ZH     3628
NL      572
FR      166
ES        3
PL        1
IT        1
Name: caller_id.u_communication_language, dtype: int64

In [135]:
keep = ['EN',
'DE',
'ZH',
'NL']

In [136]:
data = data[data['caller_id.u_communication_language'].isin(keep)]

In [137]:
data['caller_id.u_communication_language'].value_counts()

EN    21556
DE     5634
ZH     3628
NL      572
Name: caller_id.u_communication_language, dtype: int64

In [138]:
data['company.u_location.zip'].value_counts()

43240            10926
8606              5371
2822              3397
200233            2969
35396             2184
LE4 1AW           1276
400072            1018
213125             658
4004               649
06455-000          591
21046              442
MA 01730           429
40150              318
8603               212
1230               198
CA 94621-0060      143
SG8 5HN            126
43085-4766         123
8902                90
M50 2XD             85
7054                77
139959              28
5215                21
43229-1012          17
1932                13
10310                9
3364                 8
999077               5
14850                2
3207                 2
213022               1
610000               1
2459                 1
Name: company.u_location.zip, dtype: int64

In [139]:
keep = ['43240',
'8606',
'2822',
'200233',
'35396',
'LE4 1AW',
'400072',
'06455-000',
'213125',
'4004',
'21046',
'MA 01730',
'40150']

In [140]:
data = data[data['company.u_location.zip'].isin(keep)]
data['company.u_location.zip'].value_counts()

43240       10926
8606         5371
2822         3397
200233       2969
35396        2184
LE4 1AW      1276
400072       1018
213125        658
4004          649
21046         442
MA 01730      429
40150         318
Name: company.u_location.zip, dtype: int64

In [141]:
data['category'].value_counts()

General Service      16531
Application           7133
Failure               2454
Service               2356
Security              1002
Misplaced Call          68
Infrastructure          56
Hardware                25
Customer Feedback        7
software                 3
request                  2
Name: category, dtype: int64

In [142]:
keep = ['General Service',      
'Application',           
'Failure',            
'Service',
'Security']

In [143]:
data = data[data['category'].isin(keep)]
data['category'].value_counts()

General Service    16531
Application         7133
Failure             2454
Service             2356
Security            1002
Name: category, dtype: int64

In [144]:
data['escalation'].value_counts()

Normal     29355
Overdue       73
High          48
Name: escalation, dtype: int64

In [145]:
data = data.drop("escalation", axis=1)

In [146]:
data['priority'].value_counts()

5    15668
4     7372
3     6149
2      277
1       10
Name: priority, dtype: int64

In [147]:
keep = [5, 4, 3, 2]

In [148]:
data = data[data['priority'].isin(keep)]
data['priority'].value_counts()

5    15668
4     7372
3     6149
2      277
Name: priority, dtype: int64

In [149]:
data['company'].value_counts()

1005                9625
4602                3230
2805                2637
215                 2402
201                 1988
101                 1713
801                 1271
1007                1246
4005                 900
214                  806
501                  649
2801                 625
1012                 442
1010                 418
2811                 258
1401                 247
106                  241
105                  151
4603                 132
4002                 110
203                   68
1403                  66
102                   49
2807                  44
213                   26
4601                  25
103                   20
211                   18
2802                  14
0206 ?DO NOT USE      10
1011                   8
4006                   6
205                    4
CH56                   4
1405                   2
2806                   2
9991                   2
216                    1
2804                   1
204                    1


In [150]:
keep = ['1005'
'4602',
'2805',
'215',
'201',
'101',
'801',
'1007',
'4005',
'214',
'701',
'501',
'2801',
'1012',
'1010']

In [151]:
data = data[data['company'].isin(keep)]
data['company'].value_counts()

2805    2637
215     2402
201     1988
101     1713
801     1271
1007    1246
4005     900
214      806
501      649
2801     625
1012     442
1010     418
Name: company, dtype: int64

In [152]:
len(data)

15097

## Preliminary analysis

In [ ]:
#Looking at correlations

In [185]:
y

23008    0
31520    0
3999     0
15136    0
4368     0
22732    1
10334    0
1623     0
28037    0
22068    0
14462    0
31261    0
26134    0
26039    0
7513     0
17370    0
21740    0
19494    0
27439    0
3610     0
28445    0
19971    0
24114    0
17552    1
24650    0
250      0
9347     0
22855    0
20653    0
30059    0
        ..
25975    0
8875     0
8943     0
30231    0
23432    0
1472     0
13019    0
19693    0
13962    0
26664    0
607      0
8356     0
18648    0
7891     0
24848    0
23703    0
27118    0
17446    0
20387    0
15559    0
8258     0
22928    0
19251    1
3523     0
27945    0
16973    0
23096    0
4998     0
4527     0
16492    0
Name: company.u_location.zip_MA 01730, Length: 12077, dtype: uint8

## Building decision tree

In [190]:
#One-hot encoding
enc_data = pd.get_dummies(data)

In [191]:
y = enc_data.iloc[:, 3]
x = enc_data.drop("Completion Time", axis=1)

In [192]:
x, x_test, y, y_test = train_test_split(x, y, test_size=0.2)

In [193]:
train_data = lgb.Dataset(x, label = y)
test_data = lgb.Dataset(x_test, label=y_test)

In [194]:
parameters = {
    'application': 'regression',
    'objective': 'regression',
    'metric': 'l2',
    'is_unbalance': 'true',
    'boosting': 'gbdt',
    'num_leaves': 31,
    'feature_fraction': 0.5,
    'bagging_fraction': 0.5,
    'bagging_freq': 20,
    'learning_rate': 0.05,
    'verbose': 0
}


In [214]:
model_test = lgb.train(parameters,
                       train_data,
                       num_boost_round=500)

In [215]:
for idx, val in sorted(enumerate(model_test.feature_importance()), key=lambda x: x[1], reverse=True):
    print(x.columns.values[idx], val)

priority 15575
urgency 13180
impact 11619
category_Application 8846
category_General Service 8595
caller_id.u_communication_language_DE 6437
caller_id.u_communication_language_EN 6184
category_Failure 6082
company_215 5205
company.u_location.zip_8606 4889
company_214 4747
category_Service 4694
company_201 4639
company_1007 4602
company_2805 4296
company_2801 3422
caller_id.u_communication_language_ZH 3365
company_801 3331
category_Security 3320
company_101 3046
company_4005 2310
company_501 2295
company.u_location.zip_43240 2289
company.u_location.zip_200233 2144
caller_id.u_communication_language_NL 1833
company_1010 1779
company_1012 1658
company.u_location.zip_213125 1620
company.u_location.zip_35396 1584
company.u_location.zip_LE4 1AW 1572
company.u_location.zip_4004 1133
company.u_location.zip_400072 1049
company.u_location.zip_MA 01730 876
company.u_location.zip_21046 790


In [216]:
y_pred = model_test.predict(x_test)

In [217]:
from sklearn.metrics import mean_squared_error

In [218]:
mean_squared_error(y_test, y_pred)

746450.2234099628

In [219]:
data.head()

urgency  impact company category  priority  \
0        3       3     201  Failure         5   
1        3       3     201  Service         5   
2        3       3     215  Failure         5   
3        3       3     215  Failure         5   
4        3       3     201  Failure         5   

  caller_id.u_communication_language company.u_location.zip  Completion Time  
0                                 DE                   8606       726.880278  
1                                 DE                   8606      1500.878333  
2                                 EN                   8606       157.011667  
3                                 EN                   8606      1479.401667  
4                                 DE                   8606       686.404167

## Building simple Neural net

## Visualizing results and comparing